<a href="https://colab.research.google.com/github/joeygoon/humandetection/blob/master/human_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Object Detection Demo


Welcome to the object detection inference walkthrough! This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the installation instructions before you start.

## Installing necessary packages to run Tensorflow-Object Detetion_API in colab

* install protobuf and cython
* clone tensoflow models github
* build protobuf
* at the end test installation, so you must see pass 13 test in a short time

In [0]:
!git clone https://github.com/Amin-Tgz/Tensorflow-Object-Detection-API-google-colab

Cloning into 'Tensorflow-Object-Detection-API-google-colab'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
#!pip install jupyter
#!pip install matplotlib

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pil is already the newest version (5.1.0-1).
python-tk is already the newest version (2.7.15~rc1-1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
env: PYTHONPATH=/content/models/research:/content/models/research/slim

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------


## Imports


* remove check tf version , by default last version installed on colab



In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

print('tensorflow version = ',tf.__version__)

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


tensorflow version =  1.13.1


* change directory to use utils 

In [0]:
!pwd

/content/models/research


In [0]:
%cd ./object_detection/

from utils import label_map_util

from utils import visualization_utils as vis_util

/content/models/research/object_detection


In [0]:
# This is needed to display the images.
%matplotlib inline

## Model preparation

In [0]:
!unzip /content/models/research/object_detection/images.zip
!unzip /content/models/research/object_detection/training.zip
! unzip /content/models/research/object_detection/doc.zip
! unzip /content/models/research/object_detection/inference_graph.zip
! unzip /content/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.zip




Archive:  /content/models/research/object_detection/images.zip
   creating: images/test/
  inflating: images/test/000001.jpg  
  inflating: images/test/000001.xml  
  inflating: images/test/000084.jpg  
  inflating: images/test/000084.xml  
  inflating: images/test/000179.jpg  
  inflating: images/test/000179.xml  
  inflating: images/test/000300.jpg  
  inflating: images/test/000300.xml  
  inflating: images/test/000444.jpg  
  inflating: images/test/000444.xml  
  inflating: images/test/000540.jpg  
  inflating: images/test/000540.xml  
  inflating: images/test/000660.jpg  
  inflating: images/test/000660.xml  
  inflating: images/test/000900.jpg  
  inflating: images/test/000900.xml  
  inflating: images/test/001116.jpg  
  inflating: images/test/001116.xml  
  inflating: images/test/001356.jpg  
  inflating: images/test/001356.xml  
  inflating: images/test/001428.jpg  
  inflating: images/test/001428.xml  
  inflating: images/test/001608.jpg  
  inflating: images/test/001608.xml  

In [0]:
!python xml_to_csv.py
!python generate_tfrecord.py --csv_input=/content/models/research/object_detection/images/train_labels.csv --image_dir=/content/models/research/object_detection/images/train --output_path=/content/models/research/object_detection/train.record
!python generate_tfrecord.py --csv_input=/content/models/research/object_detection/images/test_labels.csv --image_dir=/content/models/research/object_detection/images/test --output_path=/content/models/research/object_detection/test.record


Successfully converted xml to csv.
Successfully converted xml to csv.
Successfully created the TFRecords: /content/models/research/object_detection/train.record
Successfully created the TFRecords: /content/models/research/object_detection/test.record


In [0]:
!pwd


/content/models/research/object_detection


In [29]:
!python train.py --logtostderr --train_dir=/content/models/research/object_detection/training/ --pipeline_config_path=/content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Scale of 0 disables reg

In [0]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix /content/models/research/object_detection/training/model.ckpt-15355 --output_directory /content/models/research/object_detection/inference_graph


Traceback (most recent call last):
  File "export_inference_graph.py", line 156, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/platform/app.py", line 125, in run
    _sys.exit(main(argv))
  File "export_inference_graph.py", line 140, in main
    text_format.Merge(f.read(), pipeline_config)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/lib/io/file_io.py", line 125, in read
    self._preread_check()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/lib/io/file_io.py", line 85, in _preread_check
    compat.as_bytes(self.__name), 1024 * 512, status)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.NotFoundError: /content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config; No such file or directory


In [26]:
!zip -r /content/file.zip /content/models/research/object_detection

  adding: content/models/research/object_detection/ (stored 0%)
  adding: content/models/research/object_detection/export_tflite_ssd_graph_lib_test.py (deflated 84%)
  adding: content/models/research/object_detection/predictors/ (stored 0%)
  adding: content/models/research/object_detection/predictors/convolutional_keras_box_predictor.py (deflated 66%)
  adding: content/models/research/object_detection/predictors/convolutional_box_predictor.py (deflated 76%)
  adding: content/models/research/object_detection/predictors/__pycache__/ (stored 0%)
  adding: content/models/research/object_detection/predictors/__pycache__/mask_rcnn_box_predictor.cpython-36.pyc (deflated 55%)
  adding: content/models/research/object_detection/predictors/__pycache__/convolutional_keras_box_predictor.cpython-36.pyc (deflated 53%)
  adding: content/models/research/object_detection/predictors/__pycache__/rfcn_box_predictor.cpython-36.pyc (deflated 53%)
  adding: content/models/research/object_detection/predictors

In [30]:
!zip -r /content/trainingfolder.zip /content/models/research/object_detection/training

  adding: content/models/research/object_detection/training/ (stored 0%)
  adding: content/models/research/object_detection/training/model.ckpt-24824.data-00000-of-00001 (deflated 6%)
  adding: content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config (deflated 65%)
  adding: content/models/research/object_detection/training/checkpoint (deflated 79%)
  adding: content/models/research/object_detection/training/pipeline.config (deflated 65%)
  adding: content/models/research/object_detection/training/model.ckpt-24824.index (deflated 72%)
  adding: content/models/research/object_detection/training/model.ckpt-31537.meta (deflated 94%)
  adding: content/models/research/object_detection/training/events.out.tfevents.1554581825.5293cabc849e (deflated 89%)
  adding: content/models/research/object_detection/training/model.ckpt-26501.meta (deflated 94%)
  adding: content/models/research/object_detection/training/events.out.tfevents.1554571905.5293cabc849e (deflated 88